三、程式設計 (32%)
題目要求： 設計猜數字遊戲 (1A2B)。

規則： 謎底為 4 位數、數字 0-9 不重複。

功能： 提示「幾個位置正確 (A)」和「幾個數字正確但位置錯誤 (B)」。

限制： 輸入檢查 (不能重複、不能有符號)、最多猜 10 次。

In [5]:
ANS=str("1234")
attempt=1
while(attempt<10):
    #is valid ?
    gus=str(input("請輸入"))
    if len(gus)!=4:
        print('len error')
        continue
    if len(set(gus))!=4:
        print("can't reapt")
        continue
    if not gus.isdigit():
        print("not digit")
        continue
    A=0
    B=0
    for i in range(4):
        if gus[i]==ANS[i]:
            A+=1
        elif gus[i] in ANS:
            B+=1
    if A==4:
        print("bingo!")
        break
    attempt+=1
    if attempt<10:
        print("%dA:%dB:" %(A,B))
    else:
        print("over attemptt limit")
    
         


    


len error
len error
len error
len error
len error
len error
len error
len error
len error
can't reapt
bingo!


In [1]:
import sys
import cv2
import numpy as np
from PySide6.QtWidgets import QApplication, QMainWindow, QLabel, QVBoxLayout, QWidget, QPushButton, QLineEdit, QHBoxLayout
from PySide6.QtCore import QThread, Signal, Qt, Slot
from PySide6.QtGui import QImage, QPixmap

class VideoThread(QThread):
    """
    負責在背景接收影像的執行緒，避免卡住 UI
    """
    change_pixmap_signal = Signal(QImage)
    
    def __init__(self, url):
        super().__init__()
        self.url = url
        self.is_running = True

    def run(self):
        # 使用 OpenCV 讀取 MJPEG 串流
        cap = cv2.VideoCapture(self.url)
        
        while self.is_running:
            ret, cv_img = cap.read()
            if ret:
                # OpenCV 是 BGR，PySide 需要 RGB，所以要轉換
                rgb_image = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
                h, w, ch = rgb_image.shape
                bytes_per_line = ch * w
                
                # 轉換成 Qt 能夠顯示的 QImage 格式
                convert_to_qt_format = QImage(rgb_image.data, w, h, bytes_per_line, QImage.Format_RGB888)
                p = convert_to_qt_format.scaled(640, 480, Qt.KeepAspectRatio)
                
                # 發送訊號給 UI
                self.change_pixmap_signal.emit(p)
            else:
                print("無法讀取影像，請確認 ESP32 IP 是否正確或相機是否過熱")
                self.msleep(100) # 休息一下再試
                
        cap.release()

    def stop(self):
        self.is_running = False
        self.wait()

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("ESP32-S3 Cam Viewer - PySide6")
        self.resize(800, 600)

        # === UI 元件初始化 ===
        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)
        self.layout = QVBoxLayout(self.central_widget)

        # 1. 輸入 IP 的區域
        self.input_layout = QHBoxLayout()
        self.ip_input = QLineEdit()
        self.ip_input.setPlaceholderText("輸入 ESP32 IP (例如: 192.168.0.123)")
        # 預設串流路徑通常是 :81/stream
        self.start_btn = QPushButton("連線")
        self.stop_btn = QPushButton("停止")
        
        self.input_layout.addWidget(self.ip_input)
        self.input_layout.addWidget(self.start_btn)
        self.input_layout.addWidget(self.stop_btn)

        # 2. 顯示影像的 Label
        self.image_label = QLabel("等待連線...")
        self.image_label.setAlignment(Qt.AlignCenter)
        self.image_label.setStyleSheet("border: 2px solid gray; background-color: black; color: white;")
        self.image_label.setMinimumSize(640, 480)

        # 加入 Layout
        self.layout.addLayout(self.input_layout)
        self.layout.addWidget(self.image_label)

        # === 邏輯連接 ===
        self.start_btn.clicked.connect(self.start_video)
        self.stop_btn.clicked.connect(self.stop_video)
        self.thread = None

    @Slot()
    def start_video(self):
        ip = self.ip_input.text().strip()
        if not ip:
            self.image_label.setText("請輸入 IP 位址！")
            return

        # 組合 ESP32 CameraWebServer 預設的串流網址
        # 注意：標準範例通常使用 port 81 進行串流
        stream_url = f"http://{ip}:81/stream"
        
        self.image_label.setText(f"嘗試連線至: {stream_url} ...")

        # 建立並啟動執行緒
        self.thread = VideoThread(stream_url)
        self.thread.change_pixmap_signal.connect(self.update_image)
        self.thread.start()
        
        self.start_btn.setEnabled(False)
        self.stop_btn.setEnabled(True)

    @Slot()
    def stop_video(self):
        if self.thread:
            self.thread.stop()
            self.thread = None
        
        self.image_label.setText("連線已停止")
        self.start_btn.setEnabled(True)
        self.stop_btn.setEnabled(False)

    @Slot(QImage)
    def update_image(self, qt_img):
        # 將接收到的 QImage 顯示在 Label 上
        self.image_label.setPixmap(QPixmap.fromImage(qt_img))

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec())

無法讀取影像，請確認 ESP32 IP 是否正確或相機是否過熱
無法讀取影像，請確認 ESP32 IP 是否正確或相機是否過熱
無法讀取影像，請確認 ESP32 IP 是否正確或相機是否過熱
無法讀取影像，請確認 ESP32 IP 是否正確或相機是否過熱
無法讀取影像，請確認 ESP32 IP 是否正確或相機是否過熱
無法讀取影像，請確認 ESP32 IP 是否正確或相機是否過熱
無法讀取影像，請確認 ESP32 IP 是否正確或相機是否過熱
無法讀取影像，請確認 ESP32 IP 是否正確或相機是否過熱
無法讀取影像，請確認 ESP32 IP 是否正確或相機是否過熱
無法讀取影像，請確認 ESP32 IP 是否正確或相機是否過熱
無法讀取影像，請確認 ESP32 IP 是否正確或相機是否過熱
無法讀取影像，請確認 ESP32 IP 是否正確或相機是否過熱
無法讀取影像，請確認 ESP32 IP 是否正確或相機是否過熱
無法讀取影像，請確認 ESP32 IP 是否正確或相機是否過熱
無法讀取影像，請確認 ESP32 IP 是否正確或相機是否過熱
無法讀取影像，請確認 ESP32 IP 是否正確或相機是否過熱
無法讀取影像，請確認 ESP32 IP 是否正確或相機是否過熱
無法讀取影像，請確認 ESP32 IP 是否正確或相機是否過熱
無法讀取影像，請確認 ESP32 IP 是否正確或相機是否過熱
無法讀取影像，請確認 ESP32 IP 是否正確或相機是否過熱
無法讀取影像，請確認 ESP32 IP 是否正確或相機是否過熱
無法讀取影像，請確認 ESP32 IP 是否正確或相機是否過熱
無法讀取影像，請確認 ESP32 IP 是否正確或相機是否過熱
無法讀取影像，請確認 ESP32 IP 是否正確或相機是否過熱
無法讀取影像，請確認 ESP32 IP 是否正確或相機是否過熱
無法讀取影像，請確認 ESP32 IP 是否正確或相機是否過熱
無法讀取影像，請確認 ESP32 IP 是否正確或相機是否過熱
無法讀取影像，請確認 ESP32 IP 是否正確或相機是否過熱
無法讀取影像，請確認 ESP32 IP 是否正確或相機是否過熱
無法讀取影像，請確認 ESP32 IP 是否正確或相機是否過熱
無法讀取影像，請確認 ESP32 IP 是否正確或相機是否過熱
無法讀取影像，請

SystemExit: 0

C:\Users\ken\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
import sys
import os
import shutil
import time
import subprocess
import re

# ⚠ 需要安裝 pyserial & PySide6
# pip install pyserial PySide6

import serial
from serial.tools import list_ports
from PySide6.QtGui import QTextCursor
from PySide6.QtCore import QTimer, Qt
from PySide6.QtWidgets import (
    QApplication, QWidget, QVBoxLayout,
    QLabel, QPushButton, QPlainTextEdit, QMessageBox
)

os.environ["PYTHONIOENCODING"] = "utf-8"

# ========= 參數設定 =========
SKETCH_DIR = "esp32s3_cam"          # arduino-cli 要求: 資料夾名 = 草稿名
SKETCH_NAME = "esp32s3_cam.ino"
SOURCE_INO = "code.ino"             # ★ 你自己的 .ino 檔
FQBN = "esp32:esp32:esp32s3"        # ★ ESP32-S3
BAUD_RATE = 115200                  
# ESP32 官方 json 網址
ESP32_URL = "https://espressif.github.io/arduino-esp32/package_esp32_index.json"
# ===========================


def get_esp32_port():
    """
    用 arduino-cli board list 找 ESP32-S3 的 COM port
    """
    try:
        result = subprocess.run(
            ["arduino-cli", "board", "list"],
            capture_output=True,
            text=True,
            encoding="utf-8",
            errors="ignore"
        )
    except FileNotFoundError:
        print("找不到 arduino-cli，請先安裝並加入 PATH。")
        return None

    lines = result.stdout.strip().split("\n")
    # 優先找有 esp32 / S3 字樣的
    for line in lines:
        if line.strip().startswith("COM") and ("ESP32" in line.upper() or "S3" in line.upper()):
            return line.split()[0]

    # 找不到就退而求其次：第一個 COM
    for line in lines:
        if line.strip().startswith("COM"):
            return line.split()[0]

    return None


def prepare_sketch():
    """
    確保有 esp32s3_cam/esp32s3_cam.ino
    """
    if not os.path.exists(SOURCE_INO):
        print(f"❌ 找不到 {SOURCE_INO}，請把 code.ino 放在同一個資料夾。")
        return False

    if not os.path.exists(SKETCH_DIR):
        os.makedirs(SKETCH_DIR, exist_ok=True)

    dest_path = os.path.join(SKETCH_DIR, SKETCH_NAME)
    shutil.copy(SOURCE_INO, dest_path)
    print(f"已將 {SOURCE_INO} 複製到 {dest_path}")
    return True


def check_and_install_core(log_func=print):
    """
    檢查是否安裝了 esp32:esp32 核心，如果沒有則自動安裝
    """
    log_func("🔍 檢查 ESP32 核心安裝狀態...")
    
    # 1. 檢查是否已安裝
    check = subprocess.run(
        ["arduino-cli", "core", "list"],
        capture_output=True, text=True, encoding="utf-8", errors="ignore"
    )
    
    if "esp32:esp32" in check.stdout:
        log_func("✅ ESP32 核心已安裝。")
        return True
    
    # 2. 如果沒安裝，開始安裝流程
    log_func("⚠️ 未偵測到 ESP32 核心，開始自動安裝 (這需要一點時間)...")
    
    # 更新 index
    log_func("📥 更新 Core Index...")
    subprocess.run(
        ["arduino-cli", "core", "update-index", "--additional-urls", ESP32_URL],
        capture_output=True, text=True, encoding="utf-8"
    )
    
    # 安裝 Core
    log_func(f"📥 下載並安裝 esp32:esp32 (來源: {ESP32_URL})...")
    install = subprocess.run(
        ["arduino-cli", "core", "install", "esp32:esp32", "--additional-urls", ESP32_URL],
        capture_output=True, text=True, encoding="utf-8"
    )
    
    if install.returncode == 0:
        log_func("✅ ESP32 核心安裝成功！")
        return True
    else:
        log_func("❌ ESP32 核心安裝失敗：\n" + install.stderr)
        return False


def compile_and_upload(port, log_func=print):
    """
    用 arduino-cli compile + upload
    """
    
    # ★ 新增：先檢查核心有沒有裝
    if not check_and_install_core(log_func):
        return False

    # compile
    log_func("=== Compile (編譯中，請稍候) ===")
    try:
        # ★ 注意：必須加上 --additional-urls 否則有時會找不到依賴
        comp = subprocess.run(
            ["arduino-cli", "compile", "--fqbn", FQBN, SKETCH_DIR, "--additional-urls", ESP32_URL],
            capture_output=True,
            text=True,
            encoding="utf-8",
            errors="ignore"
        )
    except FileNotFoundError:
        log_func("❌ 找不到 arduino-cli。")
        return False

    # log_func("compile stdout:\n" + comp.stdout) # 輸出太多可以註解掉
    if comp.stderr.strip():
        log_func("compile stderr:\n" + comp.stderr)

    if comp.returncode != 0:
        log_func("❌ compile 失敗")
        return False

    # upload
    log_func("=== Upload (上傳中) ===")
    upl = subprocess.run(
        ["arduino-cli", "upload", "-p", port, "--fqbn", FQBN, SKETCH_DIR],
        capture_output=True,
        text=True,
        encoding="utf-8",
        errors="ignore"
    )
    log_func("upload stdout:\n" + upl.stdout)
    if upl.stderr.strip():
        log_func("upload stderr:\n" + upl.stderr)

    if upl.returncode != 0:
        log_func("❌ upload 失敗")
        return False

    log_func("✅ Upload complete.")
    return True


# ========== PySide6 UI ==========

class Esp32CamWindow(QWidget):
    def __init__(self, port):
        super().__init__()
        self.port = port
        self.ser = None

        self.init_ui()
        self.init_serial()
        self.init_timer()

    def init_ui(self):
        self.setWindowTitle(f"ESP32-S3-CAM 測試工具 ({self.port})")
        layout = QVBoxLayout()

        self.label_status = QLabel("狀態：就緒")
        self.label_status.setAlignment(Qt.AlignLeft)

        self.btn_upload = QPushButton("重新編譯並上傳 code.ino")
        self.btn_upload.clicked.connect(self.on_upload_clicked)

        self.text_log = QPlainTextEdit()
        self.text_log.setReadOnly(True)
        self.text_log.setPlaceholderText("日誌區域...")

        layout.addWidget(self.label_status)
        layout.addWidget(self.btn_upload)
        layout.addWidget(self.text_log)

        self.setLayout(layout)
        self.resize(700, 400)

    def append_log(self, msg: str):
        self.text_log.appendPlainText(msg.rstrip("\n"))
        cursor = self.text_log.textCursor()
        cursor.movePosition(QTextCursor.End)
        self.text_log.setTextCursor(cursor)

    def on_upload_clicked(self):
        if not prepare_sketch():
            QMessageBox.critical(self, "錯誤", "code.ino 找不到，無法上傳。")
            return

        self.label_status.setText("狀態：正在編譯 / 上傳...")
        self.append_log("\n=== 開始流程 ===\n")

        # 關閉 Serial 以免佔用上傳 Port
        self.close_serial_temp()

        # 使用 QApplication.processEvents() 讓 UI 刷新顯示 "正在編譯"
        QApplication.processEvents()

        ok = compile_and_upload(self.port, log_func=self.append_log)
        
        if ok:
            self.label_status.setText("狀態：上傳成功，等待 ESP32-S3 重啟...")
            self.reopen_serial()
        else:
            self.label_status.setText("狀態：失敗（請看下方 log）")
            self.reopen_serial() # 失敗也要把 Serial 開回來

    def init_serial(self):
        self.reopen_serial()

    def close_serial_temp(self):
        try:
            if self.ser is not None and self.ser.is_open:
                self.ser.close()
                self.append_log("🔌 暫時關閉 Serial 以便上傳...")
        except Exception:
            pass

    def reopen_serial(self):
        try:
            if self.ser is not None and self.ser.is_open:
                self.ser.close()
        except: pass
        
        try:
            self.ser = serial.Serial(self.port, BAUD_RATE, timeout=0.1)
            self.append_log(f"✅ Serial 連線成功：{self.port}")
        except Exception as e:
            self.append_log(f"⚠️ 無法開啟 Serial (可能是上傳剛結束板子還在重啟): {e}")
            self.ser = None

    def init_timer(self):
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.read_serial)
        self.timer.start(100)

    def read_serial(self):
        if self.ser is None or not self.ser.is_open:
            return
        try:
            while self.ser.in_waiting:
                line = self.ser.readline().decode(errors="ignore").rstrip("\r\n")
                if line:
                    self.append_log(f"[ESP]: {line}")
        except Exception as e:
            self.append_log(f"[Serial Error] {e}")
            self.ser = None

    def closeEvent(self, event):
        if self.ser is not None and self.ser.is_open:
            self.ser.close()
        event.accept()


if __name__ == "__main__":
    port = get_esp32_port()
    if not port:
        print("❌ 找不到 ESP32-S3 的 COM Port，請先接好再試一次。")
        port = "COM3" # 假設
    
    print(f"Detected ESP32-S3 on {port}")

    # === 修正的部分：檢查是否已有 QApplication 實例 ===
    app = QApplication.instance()
    if app is None:
        # 如果沒有存在的實例，才建立新的
        app = QApplication(sys.argv)
    else:
        print("⚠ 偵測到現有的 QApplication，沿用舊實例")

    w = Esp32CamWindow(port)
    w.show()
    
    if prepare_sketch():
        pass 

    sys.exit(app.exec())

Detected ESP32-S3 on COM3
已將 code.ino 複製到 esp32s3_cam\esp32s3_cam.ino
已將 code.ino 複製到 esp32s3_cam\esp32s3_cam.ino


SystemExit: 0

C:\Users\ken\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
import sys
import os
import shutil
import time
import subprocess
import re

# ⚠ 需要安裝 pyserial & PySide6
# pip install pyserial PySide6

import serial
from serial.tools import list_ports
from PySide6.QtGui import QTextCursor
from PySide6.QtCore import QTimer, Qt
from PySide6.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QHBoxLayout,
    QLabel, QPushButton, QPlainTextEdit, QMessageBox, QComboBox, QGroupBox
)

os.environ["PYTHONIOENCODING"] = "utf-8"

# ========= 參數設定 =========
SKETCH_DIR = "esp32s3_cam"          
SKETCH_NAME = "esp32s3_cam.ino"
SOURCE_INO = "code.ino"             
FQBN = "esp32:esp32:esp32s3"        
BAUD_RATE = 115200                  
ESP32_URL = "https://espressif.github.io/arduino-esp32/package_esp32_index.json"
# ===========================


def prepare_sketch():
    if not os.path.exists(SOURCE_INO):
        print(f"❌ 找不到 {SOURCE_INO}，請把 code.ino 放在同一個資料夾。")
        return False
    if not os.path.exists(SKETCH_DIR):
        os.makedirs(SKETCH_DIR, exist_ok=True)
    dest_path = os.path.join(SKETCH_DIR, SKETCH_NAME)
    shutil.copy(SOURCE_INO, dest_path)
    return True


def check_and_install_core(log_func=print):
    log_func("🔍 檢查 ESP32 核心安裝狀態...")
    check = subprocess.run(["arduino-cli", "core", "list"], capture_output=True, text=True, encoding="utf-8", errors="ignore")
    if "esp32:esp32" in check.stdout:
        log_func("✅ ESP32 核心已安裝。")
        return True
    
    log_func("⚠️ 未偵測到 ESP32 核心，開始自動安裝...")
    subprocess.run(["arduino-cli", "core", "update-index", "--additional-urls", ESP32_URL])
    install = subprocess.run(["arduino-cli", "core", "install", "esp32:esp32", "--additional-urls", ESP32_URL])
    
    if install.returncode == 0:
        log_func("✅ 安裝成功！")
        return True
    else:
        log_func("❌ 安裝失敗，請檢查網路或手動下載。")
        return False


def compile_and_upload(port, log_func=print):
    if not check_and_install_core(log_func):
        return False

    log_func("=== Compile (編譯中)... ===")
    comp = subprocess.run(
        ["arduino-cli", "compile", "--fqbn", FQBN, SKETCH_DIR, "--additional-urls", ESP32_URL],
        capture_output=True, text=True, encoding="utf-8", errors="ignore"
    )

    if comp.stderr.strip():
        log_func("compile stderr:\n" + comp.stderr)
    if comp.returncode != 0:
        log_func("❌ compile 失敗")
        return False

    log_func(f"=== Upload (上傳至 {port})... ===")
    # 降低速度至 115200 以確保最高相容性
    upl = subprocess.run(
        [
            "arduino-cli", "upload", 
            "-p", port, 
            "--fqbn", FQBN, 
            "--upload-field", "upload.speed=115200", 
            SKETCH_DIR
        ],
        capture_output=True, text=True, encoding="utf-8", errors="ignore"
    )
    log_func("upload stdout:\n" + upl.stdout)
    if upl.stderr.strip():
        log_func("upload stderr:\n" + upl.stderr)

    if upl.returncode != 0:
        log_func("❌ upload 失敗")
        return False

    log_func("✅ Upload complete.")
    return True


class Esp32CamWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.ser = None
        self.current_port = None
        
        self.init_ui()
        self.refresh_ports() # 啟動時自動偵測
        self.init_timer()

    def init_ui(self):
        self.setWindowTitle("ESP32-S3-CAM 工具箱 v3 (智慧選 Port)")
        layout = QVBoxLayout()

        # --- Port 選擇區 ---
        port_group = QGroupBox("1. 選擇連接埠 (COM Port)")
        port_layout = QHBoxLayout()
        
        self.combo_ports = QComboBox()
        self.combo_ports.currentIndexChanged.connect(self.on_port_changed)
        
        self.btn_refresh = QPushButton("🔄 重新整理")
        self.btn_refresh.clicked.connect(self.refresh_ports)
        
        port_layout.addWidget(self.combo_ports, 1)
        port_layout.addWidget(self.btn_refresh)
        port_group.setLayout(port_layout)

        # --- 功能按鈕區 ---
        btn_group = QGroupBox("2. 操作功能")
        btn_layout = QHBoxLayout()
        
        self.btn_reset = QPushButton("⚡ 強制進入燒錄模式")
        self.btn_reset.setStyleSheet("background-color: #f0ad4e; color: white; font-weight: bold;")
        self.btn_reset.clicked.connect(self.force_bootloader)

        self.btn_upload = QPushButton("🔥 編譯並上傳")
        self.btn_upload.setStyleSheet("background-color: #d9534f; color: white; font-weight: bold;")
        self.btn_upload.clicked.connect(self.on_upload_clicked)
        
        btn_layout.addWidget(self.btn_reset)
        btn_layout.addWidget(self.btn_upload)
        btn_group.setLayout(btn_layout)

        # --- 日誌區 ---
        self.label_status = QLabel("狀態：請選擇 Port")
        self.text_log = QPlainTextEdit()
        self.text_log.setReadOnly(True)

        layout.addWidget(port_group)
        layout.addWidget(btn_group)
        layout.addWidget(self.label_status)
        layout.addWidget(self.text_log)

        self.setLayout(layout)
        self.resize(700, 500)

    def append_log(self, msg: str):
        self.text_log.appendPlainText(msg.rstrip("\n"))
        self.text_log.moveCursor(QTextCursor.End)

    def refresh_ports(self):
        """
        自動偵測 Port，並優先選擇 COM7 或包含 ESP32 關鍵字的 Port
        """
        self.close_serial_temp()
        self.combo_ports.blockSignals(True) # 避免觸發 change 事件
        self.combo_ports.clear()
        
        ports = list_ports.comports()
        found_ports = []
        best_index = -1
        
        for i, p in enumerate(ports):
            # 顯示格式: COM7 - USB-SERIAL CH340 (COM7)
            display_text = f"{p.device} - {p.description}"
            self.combo_ports.addItem(display_text)
            found_ports.append(p.device)
            
            # === 智慧偵測邏輯 ===
            # 1. 如果找到用戶指定的 COM7，絕對優先
            if "COM7" in p.device:
                best_index = i
            # 2. 如果還沒找到 COM7，找看看有沒有寫著 ESP32 或 CP210x/CH340 的
            elif best_index == -1:
                desc = p.description.upper()
                if "ESP32" in desc or "S3" in desc or "CP210" in desc or "CH340" in desc:
                    best_index = i
        
        self.combo_ports.blockSignals(False)
        
        if len(found_ports) > 0:
            # 如果有找到最佳候選，選它；否則選第一個
            target_index = best_index if best_index != -1 else 0
            self.combo_ports.setCurrentIndex(target_index)
            
            selected_port = found_ports[target_index]
            self.current_port = selected_port
            self.label_status.setText(f"狀態：已選擇 {selected_port}")
            self.append_log(f"🔍 自動偵測到 {len(found_ports)} 個裝置，預設選擇：{selected_port}")
            self.reopen_serial()
        else:
            self.current_port = None
            self.label_status.setText("狀態：未偵測到任何 COM Port")
            self.append_log("❌ 未找到任何 COM Port，請檢查連接線。")

    def on_port_changed(self):
        # 下拉選單改變時，更新當前 Port 並重連 Serial
        current_text = self.combo_ports.currentText()
        if current_text:
            port = current_text.split(" - ")[0] # 取出 "COM7"
            self.current_port = port
            self.label_status.setText(f"狀態：切換至 {port}")
            self.append_log(f"🔌 切換 Port 至：{port}")
            self.reopen_serial()

    def force_bootloader(self):
        if not self.current_port:
            QMessageBox.warning(self, "錯誤", "請先選擇 COM Port")
            return

        self.close_serial_temp()
        self.append_log(f"⚡ 嘗試對 {self.current_port} 發送 Boot 訊號...")
        try:
            s = serial.Serial(self.current_port, BAUD_RATE)
            s.dtr = False
            s.rts = False
            s.dtr = True; s.rts = False; time.sleep(0.1)
            s.rts = True; time.sleep(0.1)
            s.rts = False; time.sleep(0.2)
            s.dtr = False
            s.close()
            self.append_log("✅ 訊號發送完畢，請立即點擊「編譯並上傳」！")
        except Exception as e:
            self.append_log(f"❌ 無法控制 Port: {e}")
        
        self.reopen_serial()

    def on_upload_clicked(self):
        if not self.current_port:
            QMessageBox.warning(self, "錯誤", "請先選擇 COM Port")
            return
            
        if not prepare_sketch():
            return

        self.label_status.setText(f"狀態：正在上傳至 {self.current_port}...")
        self.append_log("\n=== 開始流程 ===\n")

        self.close_serial_temp()
        QApplication.processEvents()
        time.sleep(0.5)

        ok = compile_and_upload(self.current_port, log_func=self.append_log)
        
        if ok:
            self.label_status.setText("狀態：成功！")
            self.reopen_serial()
        else:
            self.label_status.setText("狀態：失敗")
            self.append_log("\n❌ 失敗提示：請嘗試在出現 'Connecting...' 時按住 BOOT 鍵！")
            self.reopen_serial()

    def close_serial_temp(self):
        try:
            if self.ser and self.ser.is_open:
                self.ser.close()
        except: pass

    def reopen_serial(self):
        if not self.current_port: return
        try:
            if self.ser and self.ser.is_open: self.ser.close()
            self.ser = serial.Serial(self.current_port, BAUD_RATE, timeout=0.1)
            self.append_log(f"🔗 Serial 已連線: {self.current_port}")
        except: 
            self.ser = None

    def init_timer(self):
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.read_serial)
        self.timer.start(100)

    def read_serial(self):
        if self.ser and self.ser.is_open and self.ser.in_waiting:
            try:
                line = self.ser.readline().decode(errors="ignore").strip()
                if line: self.append_log(f"[ESP]: {line}")
            except: pass

    def closeEvent(self, event):
        if self.ser: self.ser.close()
        event.accept()

if __name__ == "__main__":
    # 檢查 QApplication 單例
    app = QApplication.instance()
    if app is None: app = QApplication(sys.argv)

    w = Esp32CamWindow()
    w.show()
    sys.exit(app.exec())

SystemExit: 0

C:\Users\ken\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
import sys
import os
import shutil
import time
import subprocess
import re
import cv2
import numpy as np

# ⚠ 需要安裝 pyserial & PySide6 & opencv-python
# pip install pyserial PySide6 opencv-python

import serial
from serial.tools import list_ports
from PySide6.QtGui import QTextCursor, QImage, QPixmap
from PySide6.QtCore import QTimer, Qt, QThread, Signal
from PySide6.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QHBoxLayout,
    QLabel, QPushButton, QPlainTextEdit, QMessageBox, QComboBox, QGroupBox, QLineEdit
)

os.environ["PYTHONIOENCODING"] = "utf-8"

# ========= 參數設定 =========
SKETCH_DIR = "esp32s3_cam"          
SKETCH_NAME = "esp32s3_cam.ino"
SOURCE_INO = "code.ino"             
FQBN = "esp32:esp32:esp32s3"        
BAUD_RATE = 115200                  
ESP32_URL = "https://espressif.github.io/arduino-esp32/package_esp32_index.json"
# ===========================

# === 影像接收執行緒 ===
class VideoThread(QThread):
    change_pixmap_signal = Signal(QImage)
    
    def __init__(self, url):
        super().__init__()
        self.url = url
        self.is_running = True

    def run(self):
        cap = cv2.VideoCapture(self.url)
        while self.is_running:
            ret, cv_img = cap.read()
            if ret:
                rgb_image = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
                h, w, ch = rgb_image.shape
                bytes_per_line = ch * w
                convert_to_qt_format = QImage(rgb_image.data, w, h, bytes_per_line, QImage.Format_RGB888)
                p = convert_to_qt_format.scaled(640, 480, Qt.KeepAspectRatio)
                self.change_pixmap_signal.emit(p)
            else:
                self.msleep(100)
        cap.release()

    def stop(self):
        self.is_running = False
        self.wait()

# === 燒錄相關函式 ===
def prepare_sketch():
    if not os.path.exists(SOURCE_INO):
        print(f"❌ 找不到 {SOURCE_INO}，請把 code.ino 放在同一個資料夾。")
        return False
    if not os.path.exists(SKETCH_DIR):
        os.makedirs(SKETCH_DIR, exist_ok=True)
    dest_path = os.path.join(SKETCH_DIR, SKETCH_NAME)
    shutil.copy(SOURCE_INO, dest_path)
    return True

def check_and_install_core(log_func=print):
    log_func("🔍 檢查 ESP32 核心安裝狀態...")
    check = subprocess.run(["arduino-cli", "core", "list"], capture_output=True, text=True, encoding="utf-8", errors="ignore")
    if "esp32:esp32" in check.stdout:
        log_func("✅ ESP32 核心已安裝。")
        return True
    log_func("⚠️ 未偵測到 ESP32 核心，開始自動安裝...")
    subprocess.run(["arduino-cli", "core", "update-index", "--additional-urls", ESP32_URL])
    install = subprocess.run(["arduino-cli", "core", "install", "esp32:esp32", "--additional-urls", ESP32_URL])
    return install.returncode == 0

def compile_and_upload(port, log_func=print):
    if not check_and_install_core(log_func): return False

    log_func("=== Compile (編譯中)... ===")
    comp = subprocess.run(["arduino-cli", "compile", "--fqbn", FQBN, SKETCH_DIR, "--additional-urls", ESP32_URL], capture_output=True, text=True, encoding="utf-8", errors="ignore")
    if comp.returncode != 0:
        log_func("❌ compile 失敗:\n" + comp.stderr)
        return False

    log_func(f"=== Upload (上傳至 {port})... ===")
    upl = subprocess.run(["arduino-cli", "upload", "-p", port, "--fqbn", FQBN, "--upload-field", "upload.speed=460800", SKETCH_DIR], capture_output=True, text=True, encoding="utf-8", errors="ignore")
    log_func("upload stdout:\n" + upl.stdout)
    if upl.returncode != 0:
        log_func("❌ upload 失敗:\n" + upl.stderr)
        return False

    log_func("✅ Upload complete.")
    return True

# === 主視窗 ===
class Esp32CamWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.ser = None
        self.current_port = None
        self.video_thread = None
        
        self.init_ui()
        self.refresh_ports()
        self.init_timer()

    def init_ui(self):
        self.setWindowTitle("ESP32-S3-CAM 終極控制台 v4")
        layout = QVBoxLayout()

        # 1. Port 選擇
        port_group = QGroupBox("1. 硬體連線")
        port_layout = QHBoxLayout()
        self.combo_ports = QComboBox()
        self.combo_ports.currentIndexChanged.connect(self.on_port_changed)
        self.btn_refresh = QPushButton("🔄 重新整理")
        self.btn_refresh.clicked.connect(self.refresh_ports)
        port_layout.addWidget(self.combo_ports, 1)
        port_layout.addWidget(self.btn_refresh)
        port_group.setLayout(port_layout)

        # 2. 燒錄控制
        upload_group = QGroupBox("2. 韌體燒錄")
        upload_layout = QHBoxLayout()
        self.btn_reset = QPushButton("⚡ 強制 Boot")
        self.btn_reset.clicked.connect(self.force_bootloader)
        self.btn_upload = QPushButton("🔥 編譯並上傳")
        self.btn_upload.setStyleSheet("background-color: #d9534f; color: white; font-weight: bold;")
        self.btn_upload.clicked.connect(self.on_upload_clicked)
        upload_layout.addWidget(self.btn_reset)
        upload_layout.addWidget(self.btn_upload)
        upload_group.setLayout(upload_layout)

        # 3. 影像顯示區
        video_group = QGroupBox("3. 即時影像")
        video_layout = QVBoxLayout()
        
        ip_layout = QHBoxLayout()
        self.input_ip = QLineEdit()
        self.input_ip.setPlaceholderText("等待 Serial 自動抓取 IP...")
        self.btn_connect = QPushButton("📺 連線影像")
        self.btn_connect.setStyleSheet("background-color: #5bc0de; color: white; font-weight: bold;")
        self.btn_connect.clicked.connect(self.start_video)
        ip_layout.addWidget(QLabel("ESP32 IP:"))
        ip_layout.addWidget(self.input_ip)
        ip_layout.addWidget(self.btn_connect)

        self.video_label = QLabel("等待連線...")
        self.video_label.setAlignment(Qt.AlignCenter)
        self.video_label.setStyleSheet("background-color: black; color: white; border: 2px solid gray;")
        self.video_label.setMinimumSize(640, 480)

        video_layout.addLayout(ip_layout)
        video_layout.addWidget(self.video_label)
        video_group.setLayout(video_layout)

        # 4. 日誌
        self.label_status = QLabel("狀態：就緒")
        self.text_log = QPlainTextEdit()
        self.text_log.setReadOnly(True)
        self.text_log.setMaximumHeight(150)

        layout.addWidget(port_group)
        layout.addWidget(upload_group)
        layout.addWidget(video_group)
        layout.addWidget(self.label_status)
        layout.addWidget(self.text_log)
        self.setLayout(layout)
        self.resize(800, 700)

    def append_log(self, msg: str):
        self.text_log.appendPlainText(msg.rstrip("\n"))
        self.text_log.moveCursor(QTextCursor.End)

    def refresh_ports(self):
        self.close_serial_temp()
        self.combo_ports.blockSignals(True)
        self.combo_ports.clear()
        ports = list_ports.comports()
        best_index = -1
        for i, p in enumerate(ports):
            self.combo_ports.addItem(f"{p.device} - {p.description}")
            if "COM7" in p.device: best_index = i
            elif best_index == -1 and ("ESP32" in p.description.upper() or "CP210" in p.description): best_index = i
        
        self.combo_ports.blockSignals(False)
        if ports:
            self.combo_ports.setCurrentIndex(best_index if best_index != -1 else 0)
            self.on_port_changed()
        else:
            self.label_status.setText("狀態：未偵測到 Port")

    def on_port_changed(self):
        text = self.combo_ports.currentText()
        if text:
            self.current_port = text.split(" - ")[0]
            self.label_status.setText(f"已選擇 {self.current_port}")
            self.reopen_serial()

    def force_bootloader(self):
        if not self.current_port: return
        self.close_serial_temp()
        try:
            s = serial.Serial(self.current_port, BAUD_RATE)
            s.dtr = False; s.rts = False
            s.dtr = True; s.rts = False; time.sleep(0.1)
            s.rts = True; time.sleep(0.1)
            s.rts = False; time.sleep(0.2)
            s.dtr = False; s.close()
            self.append_log("✅ 強制 Boot 訊號已發送")
        except: pass
        self.reopen_serial()

    def on_upload_clicked(self):
        if not self.current_port or not prepare_sketch(): return
        self.close_serial_temp()
        QApplication.processEvents(); time.sleep(0.5)
        ok = compile_and_upload(self.current_port, log_func=self.append_log)
        self.reopen_serial()
        if ok: 
            self.label_status.setText("上傳成功！請按下板子 RESET 鍵")
            QMessageBox.information(self, "成功", "上傳成功！\n\n如果下方 Log 沒有出現 IP，\n請手動按一下板子上的 [RESET] 鍵。")

    def start_video(self):
        ip = self.input_ip.text().strip()
        if not ip: return
        url = f"http://{ip}:81/stream"
        if self.video_thread: self.video_thread.stop()
        self.video_thread = VideoThread(url)
        self.video_thread.change_pixmap_signal.connect(lambda img: self.video_label.setPixmap(QPixmap.fromImage(img)))
        self.video_thread.start()

    def close_serial_temp(self):
        try: 
            if self.ser: self.ser.close()
        except: pass

    def reopen_serial(self):
        if not self.current_port: return
        try:
            if self.ser and self.ser.is_open: self.ser.close()
            self.ser = serial.Serial(self.current_port, BAUD_RATE, timeout=0.1)
            self.append_log(f"🔗 Serial 已連線: {self.current_port}")
        except: self.ser = None

    def init_timer(self):
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.read_serial)
        self.timer.start(100)

    def read_serial(self):
        if self.ser and self.ser.is_open and self.ser.in_waiting:
            try:
                line = self.ser.readline().decode(errors="ignore").strip()
                if line: 
                    self.append_log(f"[ESP]: {line}")
                    # 自動抓取 IP
                    if "Camera Ready! Use 'http://" in line:
                        ip = line.split("http://")[1].split("'")[0]
                        self.input_ip.setText(ip)
                        self.label_status.setText(f"抓到 IP: {ip}")
                        # 自動連線
                        self.start_video()
            except: pass

    def closeEvent(self, event):
        if self.video_thread: self.video_thread.stop()
        if self.ser: self.ser.close()
        event.accept()

if __name__ == "__main__":
    app = QApplication.instance() or QApplication(sys.argv)
    w = Esp32CamWindow()
    w.show()
    sys.exit(app.exec())

SystemExit: 0

C:\Users\ken\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


claud

In [5]:
import sys
import os
import shutil
import time
import subprocess
import threading
from pathlib import Path
from typing import Optional, Callable

import cv2
import numpy as np
import serial
from serial.tools import list_ports
from PySide6.QtGui import QTextCursor, QImage, QPixmap, QFont
from PySide6.QtCore import QTimer, Qt, QThread, Signal, Slot
from PySide6.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QHBoxLayout,
    QLabel, QPushButton, QPlainTextEdit, QMessageBox, 
    QComboBox, QGroupBox, QLineEdit, QProgressBar, QCheckBox
)

os.environ["PYTHONIOENCODING"] = "utf-8"

# ========= 配置管理 =========
class Config:
    SKETCH_DIR = "esp32s3_cam"
    SKETCH_NAME = "esp32s3_cam.ino"
    SOURCE_INO = "code.ino"
    FQBN = "esp32:esp32:esp32s3"
    BAUD_RATE = 115200
    ESP32_URL = "https://espressif.github.io/arduino-esp32/package_esp32_index.json"
    VIDEO_WIDTH = 640
    VIDEO_HEIGHT = 480
    SERIAL_TIMEOUT = 0.1
    RECONNECT_INTERVAL = 200  # ms


# ========= 影像接收執行緒 =========
class VideoThread(QThread):
    """優化的影像串流執行緒，支援錯誤處理和重連"""
    change_pixmap_signal = Signal(QImage)
    error_signal = Signal(str)
    status_signal = Signal(str)
    
    def __init__(self, url: str):
        super().__init__()
        self.url = url
        self.is_running = True
        self._cap = None

    def run(self):
        retry_count = 0
        max_retries = 3
        
        while self.is_running and retry_count < max_retries:
            try:
                self._cap = cv2.VideoCapture(self.url)
                
                if not self._cap.isOpened():
                    retry_count += 1
                    self.error_signal.emit(f"無法連線到影像串流 (嘗試 {retry_count}/{max_retries})")
                    self.msleep(1000)
                    continue
                
                self.status_signal.emit("影像串流已連線")
                retry_count = 0  # 重置重試計數
                
                while self.is_running:
                    ret, cv_img = self._cap.read()
                    if ret:
                        # 轉換為 RGB
                        rgb_image = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
                        h, w, ch = rgb_image.shape
                        bytes_per_line = ch * w
                        
                        # 轉換為 Qt 格式
                        qt_image = QImage(
                            rgb_image.data, w, h, 
                            bytes_per_line, QImage.Format_RGB888
                        )
                        scaled = qt_image.scaled(
                            Config.VIDEO_WIDTH, Config.VIDEO_HEIGHT, 
                            Qt.KeepAspectRatio, Qt.SmoothTransformation
                        )
                        self.change_pixmap_signal.emit(scaled)
                    else:
                        self.msleep(50)
                        # 檢查連線是否斷開
                        if not self._cap.isOpened():
                            self.error_signal.emit("影像串流中斷")
                            break
                            
            except Exception as e:
                self.error_signal.emit(f"影像串流錯誤: {str(e)}")
                retry_count += 1
                self.msleep(1000)
            finally:
                if self._cap:
                    self._cap.release()
                    
        if retry_count >= max_retries:
            self.error_signal.emit("影像串流連線失敗，已達最大重試次數")

    def stop(self):
        self.is_running = False
        if self._cap:
            self._cap.release()
        self.wait(2000)


# ========= 序列埠讀取執行緒 =========
class SerialReader(QThread):
    """獨立的序列埠讀取執行緒，避免阻塞 UI"""
    data_received = Signal(str)
    ip_detected = Signal(str)
    error_signal = Signal(str)
    
    def __init__(self, serial_port: serial.Serial):
        super().__init__()
        self.serial_port = serial_port
        self.is_running = True
    
    def run(self):
        while self.is_running and self.serial_port and self.serial_port.is_open:
            try:
                if self.serial_port.in_waiting:
                    line = self.serial_port.readline().decode(errors="ignore").strip()
                    if line:
                        self.data_received.emit(line)
                        
                        # 自動偵測 IP
                        if "Camera Ready! Use 'http://" in line:
                            ip = line.split("http://")[1].split("'")[0]
                            self.ip_detected.emit(ip)
                else:
                    self.msleep(50)
            except Exception as e:
                self.error_signal.emit(f"Serial 讀取錯誤: {str(e)}")
                break
    
    def stop(self):
        self.is_running = False
        self.wait(1000)


# ========= Arduino CLI 管理器 =========
class ArduinoManager:
    """處理 Arduino CLI 相關操作"""
    
    @staticmethod
    def prepare_sketch() -> bool:
        """準備 sketch 檔案"""
        if not Path(Config.SOURCE_INO).exists():
            return False
        
        sketch_dir = Path(Config.SKETCH_DIR)
        sketch_dir.mkdir(exist_ok=True)
        
        dest_path = sketch_dir / Config.SKETCH_NAME
        shutil.copy(Config.SOURCE_INO, dest_path)
        return True
    
    @staticmethod
    def check_and_install_core(log_func: Callable[[str], None]) -> bool:
        """檢查並安裝 ESP32 核心"""
        log_func("🔍 檢查 ESP32 核心...")
        
        try:
            result = subprocess.run(
                ["arduino-cli", "core", "list"],
                capture_output=True, text=True, 
                encoding="utf-8", errors="ignore",
                timeout=10
            )
            
            if "esp32:esp32" in result.stdout:
                log_func("✅ ESP32 核心已安裝")
                return True
            
            log_func("⚠️ 開始安裝 ESP32 核心...")
            
            # 更新索引
            subprocess.run(
                ["arduino-cli", "core", "update-index", 
                 "--additional-urls", Config.ESP32_URL],
                timeout=60
            )
            
            # 安裝核心
            result = subprocess.run(
                ["arduino-cli", "core", "install", "esp32:esp32",
                 "--additional-urls", Config.ESP32_URL],
                capture_output=True, text=True,
                encoding="utf-8", errors="ignore",
                timeout=300
            )
            
            success = result.returncode == 0
            if success:
                log_func("✅ ESP32 核心安裝完成")
            else:
                log_func(f"❌ 安裝失敗: {result.stderr}")
            
            return success
            
        except subprocess.TimeoutExpired:
            log_func("❌ 操作超時")
            return False
        except Exception as e:
            log_func(f"❌ 錯誤: {str(e)}")
            return False
    
    @staticmethod
    def compile_sketch(log_func: Callable[[str], None]) -> bool:
        """編譯 sketch"""
        log_func("=== 開始編譯 ===")
        
        try:
            result = subprocess.run(
                ["arduino-cli", "compile", 
                 "--fqbn", Config.FQBN, 
                 Config.SKETCH_DIR,
                 "--additional-urls", Config.ESP32_URL],
                capture_output=True, text=True,
                encoding="utf-8", errors="ignore",
                timeout=120
            )
            
            if result.returncode == 0:
                log_func("✅ 編譯成功")
                return True
            else:
                log_func(f"❌ 編譯失敗:\n{result.stderr}")
                return False
                
        except Exception as e:
            log_func(f"❌ 編譯錯誤: {str(e)}")
            return False
    
    @staticmethod
    def upload_sketch(port: str, log_func: Callable[[str], None]) -> bool:
        """上傳 sketch"""
        log_func(f"=== 開始上傳到 {port} ===")
        
        try:
            result = subprocess.run(
                ["arduino-cli", "upload", 
                 "-p", port,
                 "--fqbn", Config.FQBN,
                 "--upload-field", "upload.speed=460800",
                 Config.SKETCH_DIR],
                capture_output=True, text=True,
                encoding="utf-8", errors="ignore",
                timeout=60
            )
            
            log_func(result.stdout)
            
            if result.returncode == 0:
                log_func("✅ 上傳成功")
                return True
            else:
                log_func(f"❌ 上傳失敗:\n{result.stderr}")
                return False
                
        except Exception as e:
            log_func(f"❌ 上傳錯誤: {str(e)}")
            return False


# ========= 編譯上傳執行緒 =========
class CompileUploadThread(QThread):
    """在背景執行編譯上傳，避免凍結 UI"""
    progress_signal = Signal(str)
    finished_signal = Signal(bool)
    
    def __init__(self, port: str):
        super().__init__()
        self.port = port
    
    def run(self):
        # 檢查並安裝核心
        if not ArduinoManager.check_and_install_core(self.progress_signal.emit):
            self.finished_signal.emit(False)
            return
        
        # 編譯
        if not ArduinoManager.compile_sketch(self.progress_signal.emit):
            self.finished_signal.emit(False)
            return
        
        # 上傳
        success = ArduinoManager.upload_sketch(self.port, self.progress_signal.emit)
        self.finished_signal.emit(success)


# ========= 主視窗 =========
class Esp32CamWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.ser: Optional[serial.Serial] = None
        self.current_port: Optional[str] = None
        self.video_thread: Optional[VideoThread] = None
        self.serial_reader: Optional[SerialReader] = None
        self.compile_thread: Optional[CompileUploadThread] = None
        
        self.init_ui()
        self.refresh_ports()

    def init_ui(self):
        self.setWindowTitle("ESP32-S3-CAM 優化控制台 v5")
        main_layout = QVBoxLayout()
        
        # === 1. 硬體連線區 ===
        port_group = QGroupBox("1. 硬體連線")
        port_layout = QHBoxLayout()
        
        self.combo_ports = QComboBox()
        self.combo_ports.setMinimumWidth(300)
        self.combo_ports.currentIndexChanged.connect(self.on_port_changed)
        
        self.btn_refresh = QPushButton("🔄 重新整理")
        self.btn_refresh.clicked.connect(self.refresh_ports)
        self.btn_refresh.setMaximumWidth(120)
        
        self.check_auto_connect = QCheckBox("自動連線")
        self.check_auto_connect.setChecked(True)
        
        port_layout.addWidget(QLabel("連接埠:"))
        port_layout.addWidget(self.combo_ports, 1)
        port_layout.addWidget(self.check_auto_connect)
        port_layout.addWidget(self.btn_refresh)
        port_group.setLayout(port_layout)
        
        # === 2. 韌體燒錄區 ===
        upload_group = QGroupBox("2. 韌體燒錄")
        upload_layout = QVBoxLayout()
        
        btn_layout = QHBoxLayout()
        self.btn_reset = QPushButton("⚡ 強制 Boot 模式")
        self.btn_reset.clicked.connect(self.force_bootloader)
        self.btn_reset.setMaximumWidth(150)
        
        self.btn_upload = QPushButton("🔥 編譯並上傳")
        self.btn_upload.setStyleSheet(
            "background-color: #d9534f; color: white; "
            "font-weight: bold; padding: 8px;"
        )
        self.btn_upload.clicked.connect(self.on_upload_clicked)
        
        btn_layout.addWidget(self.btn_reset)
        btn_layout.addWidget(self.btn_upload, 1)
        
        self.progress_bar = QProgressBar()
        self.progress_bar.setVisible(False)
        self.progress_bar.setMaximum(0)  # 不確定進度模式
        
        upload_layout.addLayout(btn_layout)
        upload_layout.addWidget(self.progress_bar)
        upload_group.setLayout(upload_layout)
        
        # === 3. 即時影像區 ===
        video_group = QGroupBox("3. 即時影像串流")
        video_layout = QVBoxLayout()
        
        # IP 輸入列
        ip_layout = QHBoxLayout()
        self.input_ip = QLineEdit()
        self.input_ip.setPlaceholderText("等待自動偵測 IP 或手動輸入...")
        
        self.btn_connect = QPushButton("📺 開始串流")
        self.btn_connect.setStyleSheet(
            "background-color: #5bc0de; color: white; "
            "font-weight: bold; padding: 8px;"
        )
        self.btn_connect.clicked.connect(self.toggle_video)
        self.btn_connect.setMaximumWidth(150)
        
        ip_layout.addWidget(QLabel("ESP32 IP:"))
        ip_layout.addWidget(self.input_ip, 1)
        ip_layout.addWidget(self.btn_connect)
        
        # 影像顯示
        self.video_label = QLabel("等待連線...")
        self.video_label.setAlignment(Qt.AlignCenter)
        self.video_label.setStyleSheet(
            "background-color: #2c3e50; color: #ecf0f1; "
            "border: 2px solid #34495e; font-size: 14px;"
        )
        self.video_label.setMinimumSize(Config.VIDEO_WIDTH, Config.VIDEO_HEIGHT)
        
        video_layout.addLayout(ip_layout)
        video_layout.addWidget(self.video_label)
        video_group.setLayout(video_layout)
        
        # === 4. 狀態與日誌區 ===
        log_group = QGroupBox("4. 系統日誌")
        log_layout = QVBoxLayout()
        
        self.label_status = QLabel("狀態：就緒")
        self.label_status.setStyleSheet("font-weight: bold; color: #27ae60;")
        
        self.text_log = QPlainTextEdit()
        self.text_log.setReadOnly(True)
        self.text_log.setMaximumHeight(180)
        self.text_log.setFont(QFont("Consolas", 9))
        
        btn_clear = QPushButton("🗑️ 清除日誌")
        btn_clear.clicked.connect(self.text_log.clear)
        btn_clear.setMaximumWidth(120)
        
        log_layout.addWidget(self.label_status)
        log_layout.addWidget(self.text_log)
        log_layout.addWidget(btn_clear, alignment=Qt.AlignRight)
        log_group.setLayout(log_layout)
        
        # 組合所有區塊
        main_layout.addWidget(port_group)
        main_layout.addWidget(upload_group)
        main_layout.addWidget(video_group, 1)
        main_layout.addWidget(log_group)
        
        self.setLayout(main_layout)
        self.resize(900, 850)

    # === 日誌相關 ===
    @Slot(str)
    def append_log(self, msg: str):
        """安全地新增日誌（執行緒安全）"""
        timestamp = time.strftime("%H:%M:%S")
        self.text_log.appendPlainText(f"[{timestamp}] {msg.rstrip()}")
        self.text_log.moveCursor(QTextCursor.End)

    def set_status(self, msg: str, color: str = "#27ae60"):
        """更新狀態列"""
        self.label_status.setText(f"狀態：{msg}")
        self.label_status.setStyleSheet(f"font-weight: bold; color: {color};")

    # === 連接埠管理 ===
    def refresh_ports(self):
        """重新掃描並更新連接埠列表"""
        self.close_serial()
        
        self.combo_ports.blockSignals(True)
        self.combo_ports.clear()
        
        ports = list(list_ports.comports())
        if not ports:
            self.combo_ports.addItem("未偵測到裝置")
            self.set_status("未偵測到任何連接埠", "#e74c3c")
            self.combo_ports.blockSignals(False)
            return
        
        best_index = -1
        for i, port in enumerate(ports):
            desc = f"{port.device} - {port.description}"
            self.combo_ports.addItem(desc)
            
            # 智慧選擇最佳連接埠
            if "COM7" in port.device:
                best_index = i
            elif best_index == -1 and any(keyword in port.description.upper() 
                for keyword in ["ESP32", "CP210", "CH340", "USB-SERIAL"]):
                best_index = i
        
        self.combo_ports.blockSignals(False)
        
        if best_index != -1:
            self.combo_ports.setCurrentIndex(best_index)
        
        self.on_port_changed()
        self.append_log(f"🔍 發現 {len(ports)} 個連接埠")

    @Slot()
    def on_port_changed(self):
        """連接埠變更時的處理"""
        text = self.combo_ports.currentText()
        if not text or text == "未偵測到裝置":
            self.current_port = None
            return
        
        self.current_port = text.split(" - ")[0]
        self.set_status(f"已選擇 {self.current_port}")
        
        if self.check_auto_connect.isChecked():
            self.open_serial()

    def open_serial(self):
        """開啟序列埠連線"""
        if not self.current_port:
            return
        
        try:
            # 關閉舊連線
            self.close_serial()
            
            # 建立新連線
            self.ser = serial.Serial(
                self.current_port, 
                Config.BAUD_RATE, 
                timeout=Config.SERIAL_TIMEOUT
            )
            
            # 啟動讀取執行緒
            self.serial_reader = SerialReader(self.ser)
            self.serial_reader.data_received.connect(self.on_serial_data)
            self.serial_reader.ip_detected.connect(self.on_ip_detected)
            self.serial_reader.error_signal.connect(self.append_log)
            self.serial_reader.start()
            
            self.append_log(f"🔗 Serial 已連線: {self.current_port}")
            self.set_status(f"已連線 {self.current_port}", "#27ae60")
            
        except serial.SerialException as e:
            self.append_log(f"❌ Serial 連線失敗: {str(e)}")
            self.set_status("連線失敗", "#e74c3c")
            self.ser = None

    def close_serial(self):
        """關閉序列埠連線"""
        if self.serial_reader:
            self.serial_reader.stop()
            self.serial_reader = None
        
        if self.ser:
            try:
                self.ser.close()
            except:
                pass
            self.ser = None

    @Slot(str)
    def on_serial_data(self, data: str):
        """處理接收到的序列資料"""
        self.append_log(f"[ESP32] {data}")

    @Slot(str)
    def on_ip_detected(self, ip: str):
        """處理偵測到的 IP"""
        self.input_ip.setText(ip)
        self.set_status(f"偵測到 IP: {ip}", "#3498db")
        self.append_log(f"✅ 自動偵測到 IP: {ip}")
        
        # 自動開始串流
        if self.check_auto_connect.isChecked():
            QTimer.singleShot(1000, self.start_video)

    # === 韌體燒錄 ===
    @Slot()
    def force_bootloader(self):
        """強制進入 Bootloader 模式"""
        if not self.current_port:
            QMessageBox.warning(self, "警告", "請先選擇連接埠")
            return
        
        self.close_serial()
        
        try:
            s = serial.Serial(self.current_port, Config.BAUD_RATE)
            s.dtr = False
            s.rts = False
            time.sleep(0.1)
            
            s.dtr = True
            s.rts = False
            time.sleep(0.1)
            
            s.rts = True
            time.sleep(0.1)
            
            s.rts = False
            time.sleep(0.2)
            
            s.dtr = False
            s.close()
            
            self.append_log("✅ Boot 訊號已發送")
            QTimer.singleShot(500, self.open_serial)
            
        except Exception as e:
            self.append_log(f"❌ Boot 失敗: {str(e)}")

    @Slot()
    def on_upload_clicked(self):
        """開始編譯上傳流程"""
        if not self.current_port:
            QMessageBox.warning(self, "警告", "請先選擇連接埠")
            return
        
        if not ArduinoManager.prepare_sketch():
            QMessageBox.critical(
                self, "錯誤", 
                f"找不到 {Config.SOURCE_INO}\n請確認檔案存在於程式目錄"
            )
            return
        
        # 停用按鈕，顯示進度條
        self.btn_upload.setEnabled(False)
        self.btn_reset.setEnabled(False)
        self.progress_bar.setVisible(True)
        self.close_serial()
        
        # 啟動編譯上傳執行緒
        self.compile_thread = CompileUploadThread(self.current_port)
        self.compile_thread.progress_signal.connect(self.append_log)
        self.compile_thread.finished_signal.connect(self.on_upload_finished)
        self.compile_thread.start()

    @Slot(bool)
    def on_upload_finished(self, success: bool):
        """編譯上傳完成回調"""
        self.btn_upload.setEnabled(True)
        self.btn_reset.setEnabled(True)
        self.progress_bar.setVisible(False)
        
        if success:
            self.set_status("上傳成功！", "#27ae60")
            QMessageBox.information(
                self, "成功", 
                "韌體上傳成功！\n\n"
                "如果 Log 沒有顯示 IP，\n"
                "請手動按下板子上的 RESET 鍵。"
            )
            QTimer.singleShot(1000, self.open_serial)
        else:
            self.set_status("上傳失敗", "#e74c3c")
            QMessageBox.critical(
                self, "失敗",
                "上傳失敗！\n請檢查 Log 了解詳情。"
            )

    # === 影像串流 ===
    @Slot()
    def toggle_video(self):
        """切換影像串流狀態"""
        if self.video_thread and self.video_thread.isRunning():
            self.stop_video()
        else:
            self.start_video()

    def start_video(self):
        """開始影像串流"""
        ip = self.input_ip.text().strip()
        if not ip:
            QMessageBox.warning(self, "警告", "請輸入 ESP32 的 IP 位址")
            return
        
        # 停止舊的串流
        if self.video_thread:
            self.video_thread.stop()
        
        url = f"http://{ip}:81/stream"
        self.append_log(f"📺 連線到: {url}")
        
        self.video_thread = VideoThread(url)
        self.video_thread.change_pixmap_signal.connect(self.update_video)
        self.video_thread.error_signal.connect(self.on_video_error)
        self.video_thread.status_signal.connect(self.on_video_status)
        self.video_thread.start()
        
        self.btn_connect.setText("⏸️ 停止串流")
        self.btn_connect.setStyleSheet(
            "background-color: #e67e22; color: white; "
            "font-weight: bold; padding: 8px;"
        )

    def stop_video(self):
        """停止影像串流"""
        if self.video_thread:
            self.video_thread.stop()
            self.video_thread = None
        
        self.video_label.setText("串流已停止")
        self.video_label.setPixmap(QPixmap())
        
        self.btn_connect.setText("📺 開始串流")
        self.btn_connect.setStyleSheet(
            "background-color: #5bc0de; color: white; "
            "font-weight: bold; padding: 8px;"
        )
        self.append_log("⏸️ 影像串流已停止")

    @Slot(QImage)
    def update_video(self, image: QImage):
        """更新影像顯示"""
        self.video_label.setPixmap(QPixmap.fromImage(image))

    @Slot(str)
    def on_video_error(self, error: str):
        """處理影像串流錯誤"""
        self.append_log(f"❌ {error}")
        self.video_label.setText(f"錯誤：{error}")

    @Slot(str)
    def on_video_status(self, status: str):
        """處理影像串流狀態"""
        self.set_status(status, "#3498db")

    # === 清理 ===
    def closeEvent(self, event):
        """視窗關閉時的清理"""
        self.stop_video()
        self.close_serial()
        
        if self.compile_thread and self.compile_thread.isRunning():
            self.compile_thread.wait(2000)
        
        event.accept()


# ========= 程式進入點 =========
def main():
    app = QApplication.instance() or QApplication(sys.argv)
    app.setStyle("Fusion")  # 使用 Fusion 風格
    
    window = Esp32CamWindow()
    window.show()
    
    sys.exit(app.exec())


if __name__ == "__main__":
    main()

SystemExit: 0

In [1]:
import sys
import os
import shutil
import time
import subprocess
import re
import cv2
import numpy as np

# ⚠ 需要安裝 pyserial & PySide6 & opencv-python
# pip install pyserial PySide6 opencv-python

import serial
from serial.tools import list_ports
from PySide6.QtGui import QTextCursor, QImage, QPixmap
from PySide6.QtCore import QTimer, Qt, QThread, Signal
from PySide6.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QHBoxLayout,
    QLabel, QPushButton, QPlainTextEdit, QMessageBox, QComboBox, QGroupBox, QLineEdit
)

os.environ["PYTHONIOENCODING"] = "utf-8"

# ========= 參數設定 =========
SKETCH_DIR = "esp32s3_cam"          
SKETCH_NAME = "esp32s3_cam.ino"
SOURCE_INO = "code.ino"             
FQBN = (
    "esp32:esp32:esp32s3:"
    "FlashSize=16M,"          # 16MB Flash
    "PSRAM=opi,"              # OPI PSRAM（對應 Octal PSRAM）
    "PartitionScheme=fatflash,"  # 16M 最常用：2MB APP + 12.5MB FATFS
    "UploadSpeed=921600"      # 跟你剛剛顯示的一樣
)


BAUD_RATE = 115200                  
ESP32_URL = "https://espressif.github.io/arduino-esp32/package_esp32_index.json"
# ===========================

# === 影像接收執行緒 ===
class VideoThread(QThread):
    change_pixmap_signal = Signal(QImage)
    
    def __init__(self, url):
        super().__init__()
        self.url = url
        self.is_running = True

    def run(self):
        cap = cv2.VideoCapture(self.url)
        while self.is_running:
            ret, cv_img = cap.read()
            if ret:
                rgb_image = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
                h, w, ch = rgb_image.shape
                bytes_per_line = ch * w
                convert_to_qt_format = QImage(rgb_image.data, w, h, bytes_per_line, QImage.Format_RGB888)
                p = convert_to_qt_format.scaled(640, 480, Qt.KeepAspectRatio)
                self.change_pixmap_signal.emit(p)
            else:
                self.msleep(100)
        cap.release()

    def stop(self):
        self.is_running = False
        self.wait()

# === 燒錄相關函式 ===
def prepare_sketch():
    if not os.path.exists(SOURCE_INO):
        print(f"❌ 找不到 {SOURCE_INO}，請把 code.ino 放在同一個資料夾。")
        return False
    if not os.path.exists(SKETCH_DIR):
        os.makedirs(SKETCH_DIR, exist_ok=True)
    dest_path = os.path.join(SKETCH_DIR, SKETCH_NAME)
    shutil.copy(SOURCE_INO, dest_path)
    return True

def check_and_install_core(log_func=print):
    log_func("🔍 檢查 ESP32 核心安裝狀態...")
    check = subprocess.run(["arduino-cli", "core", "list"], capture_output=True, text=True, encoding="utf-8", errors="ignore")
    if "esp32:esp32" in check.stdout:
        log_func("✅ ESP32 核心已安裝。")
        return True
    log_func("⚠️ 未偵測到 ESP32 核心，開始自動安裝...")
    subprocess.run(["arduino-cli", "core", "update-index", "--additional-urls", ESP32_URL])
    install = subprocess.run(["arduino-cli", "core", "install", "esp32:esp32", "--additional-urls", ESP32_URL])
    return install.returncode == 0

def compile_and_upload(port, log_func=print):
    if not check_and_install_core(log_func): return False

    log_func("=== Compile (編譯中)... ===")
    comp = subprocess.run(["arduino-cli", "compile", "--fqbn", FQBN, SKETCH_DIR, "--additional-urls", ESP32_URL], capture_output=True, text=True, encoding="utf-8", errors="ignore")
    if comp.returncode != 0:
        log_func("❌ compile 失敗:\n" + comp.stderr)
        return False

    log_func(f"=== Upload (上傳至 {port})... ===")
    
    upl = subprocess.run(
        [
            "arduino-cli", "upload", 
            "-p", port, 
            "--fqbn", FQBN, 
            "--upload-field", "upload.speed=115200",
            "--upload-field", "upload.flash_mode=dio",
            SKETCH_DIR
        ],
        capture_output=True, text=True, encoding="utf-8", errors="ignore"
    )
    log_func("upload stdout:\n" + upl.stdout)
    if upl.returncode != 0:
        log_func("❌ upload 失敗:\n" + upl.stderr)
        return False

    log_func("✅ Upload complete.")
    return True

# === 主視窗 ===
class Esp32CamWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.ser = None
        self.current_port = None
        self.video_thread = None
        self.serial_enabled = True  # 控制是否讀取 Serial
        
        self.init_ui()
        self.refresh_ports()
        self.init_timer()

    def init_ui(self):
        self.setWindowTitle("ESP32-S3-CAM N16R8 穩定版 v5.0")
        layout = QVBoxLayout()

        # 1. Port 選擇
        port_group = QGroupBox("1. 硬體連線")
        port_layout = QHBoxLayout()
        self.combo_ports = QComboBox()
        self.combo_ports.currentIndexChanged.connect(self.on_port_changed)
        self.btn_refresh = QPushButton("🔄 重新整理")
        self.btn_refresh.clicked.connect(self.refresh_ports)
        port_layout.addWidget(self.combo_ports, 1)
        port_layout.addWidget(self.btn_refresh)
        port_group.setLayout(port_layout)

        # 2. 燒錄控制
        upload_group = QGroupBox("2. 韌體燒錄")
        upload_layout = QHBoxLayout()
        self.btn_reset = QPushButton("⚡ 強制 Boot")
        self.btn_reset.clicked.connect(self.force_bootloader)
        self.btn_upload = QPushButton("🔥 上傳韌體")
        self.btn_upload.setStyleSheet("background-color: #d9534f; color: white; font-weight: bold;")
        self.btn_upload.clicked.connect(self.on_upload_clicked)
        upload_layout.addWidget(self.btn_reset)
        upload_layout.addWidget(self.btn_upload)
        upload_group.setLayout(upload_layout)

        # 3. 影像顯示區
        video_group = QGroupBox("3. 即時影像")
        video_layout = QVBoxLayout()
        
        ip_layout = QHBoxLayout()
        self.input_ip = QLineEdit()
        self.input_ip.setPlaceholderText("等待 Serial 自動抓取 IP...")
        self.btn_connect = QPushButton("📺 連線影像")
        self.btn_connect.setStyleSheet("background-color: #5bc0de; color: white; font-weight: bold;")
        self.btn_connect.clicked.connect(self.start_video)
        ip_layout.addWidget(QLabel("ESP32 IP:"))
        ip_layout.addWidget(self.input_ip)
        ip_layout.addWidget(self.btn_connect)

        self.video_label = QLabel("等待連線...")
        self.video_label.setAlignment(Qt.AlignCenter)
        self.video_label.setStyleSheet("background-color: black; color: white; border: 2px solid gray;")
        self.video_label.setMinimumSize(640, 480)

        video_layout.addLayout(ip_layout)
        video_layout.addWidget(self.video_label)
        video_group.setLayout(video_layout)

        # 4. 日誌
        self.label_status = QLabel("狀態：就緒")
        self.text_log = QPlainTextEdit()
        self.text_log.setReadOnly(True)
        self.text_log.setMaximumHeight(150)

        layout.addWidget(port_group)
        layout.addWidget(upload_group)
        layout.addWidget(video_group)
        layout.addWidget(self.label_status)
        layout.addWidget(self.text_log)
        self.setLayout(layout)
        self.resize(800, 700)

    def append_log(self, msg: str):
        self.text_log.appendPlainText(msg.rstrip("\n"))
        self.text_log.moveCursor(QTextCursor.End)

    def refresh_ports(self):
        self.close_serial_safe()
        self.combo_ports.blockSignals(True)
        self.combo_ports.clear()
        ports = list_ports.comports()
        best_index = -1
        for i, p in enumerate(ports):
            self.combo_ports.addItem(f"{p.device} - {p.description}")
            if "COM7" in p.device: best_index = i
            elif best_index == -1 and ("ESP32" in p.description.upper() or "CP210" in p.description): best_index = i
        
        self.combo_ports.blockSignals(False)
        if ports:
            self.combo_ports.setCurrentIndex(best_index if best_index != -1 else 0)
            self.on_port_changed()
        else:
            self.label_status.setText("狀態：未偵測到 Port")

    def on_port_changed(self):
        text = self.combo_ports.currentText()
        if text:
            self.current_port = text.split(" - ")[0]
            self.label_status.setText(f"已選擇 {self.current_port}")
            self.reopen_serial()

    def force_bootloader(self):
        if not self.current_port: return
        self.close_serial_safe()
        try:
            s = serial.Serial(self.current_port, BAUD_RATE)
            s.dtr = False; s.rts = False
            s.dtr = True; s.rts = False; time.sleep(0.1)
            s.rts = True; time.sleep(0.1)
            s.rts = False; time.sleep(0.2)
            s.dtr = False; s.close()
            self.append_log("✅ 強制 Boot 訊號已發送")
        except Exception as e:
            self.append_log(f"⚠️  Boot 訊號發送失敗: {e}")
        time.sleep(0.5)
        self.reopen_serial()

    def on_upload_clicked(self):
        if not self.current_port or not prepare_sketch(): return
        
        # ★ 關鍵修復：上傳前完全關閉 Serial
        self.serial_enabled = False
        self.close_serial_safe()
        self.append_log("🔒 已關閉 Serial 監視器，準備上傳...")
        
        QApplication.processEvents()
        time.sleep(1.0)  # 確保 Port 釋放
        
        ok = compile_and_upload(self.current_port, log_func=self.append_log)
        
        # 上傳完成後等待 3 秒再重新開啟
        time.sleep(3.0)
        self.serial_enabled = True
        self.reopen_serial()
        
        if ok: 
            self.label_status.setText("✅ 上傳成功！正在重啟...")
            self.append_log("\n✅ 韌體已更新，請觀察下方 Log...")
            QMessageBox.information(self, "成功", 
                "✅ 上傳成功！\n\n"
                "ESP32 正在重啟中...\n"
                "請觀察下方 Log 是否出現診斷資訊。\n\n"
                "如果 5 秒後沒有反應，請按板子上的 RESET 鍵。")
        else:
            self.label_status.setText("❌ 上傳失敗")
            self.append_log("\n❌ 上傳失敗，請檢查錯誤訊息")

    def start_video(self):
        ip = self.input_ip.text().strip()
        if not ip: 
            QMessageBox.warning(self, "提示", "請先等待 ESP32 開機並取得 IP 位址")
            return
        
        url = f"http://{ip}:81/stream"
        self.append_log(f"📺 正在連接: {url}")
        
        if self.video_thread: 
            self.video_thread.stop()
        
        self.video_thread = VideoThread(url)
        self.video_thread.change_pixmap_signal.connect(
            lambda img: self.video_label.setPixmap(QPixmap.fromImage(img))
        )
        self.video_thread.start()
        self.label_status.setText(f"📺 影像串流中: {ip}")

    def close_serial_safe(self):
        """安全關閉 Serial，包含異常處理"""
        try: 
            if self.ser and self.ser.is_open: 
                self.ser.close()
                self.append_log("🔓 Serial Port 已釋放")
        except Exception as e:
            self.append_log(f"⚠️  關閉 Serial 時發生錯誤（可忽略）: {e}")
        finally:
            self.ser = None

    def reopen_serial(self):
        """重新開啟 Serial"""
        if not self.current_port or not self.serial_enabled: 
            return
        
        try:
            if self.ser and self.ser.is_open: 
                self.ser.close()
                time.sleep(0.2)
            
            self.ser = serial.Serial(self.current_port, BAUD_RATE, timeout=0.1)
            self.append_log(f"🔗 Serial 已連線: {self.current_port}")
            self.label_status.setText(f"🔗 Serial 監視中: {self.current_port}")
        except Exception as e:
            self.ser = None
            self.append_log(f"⚠️  無法開啟 Serial: {e}")

    def init_timer(self):
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.read_serial)
        self.timer.start(100)

    def read_serial(self):
        """安全讀取 Serial，包含異常處理"""
        if not self.serial_enabled or not self.ser:
            return
        
        try:
            if self.ser.is_open and self.ser.in_waiting:
                line = self.ser.readline().decode(errors="ignore").strip()
                if line: 
                    self.append_log(f"[ESP]: {line}")
                    
                    # 自動抓取 IP
                    if "IP 位址:" in line or "IP address:" in line:
                        # 匹配 IP 格式：xxx.xxx.xxx.xxx
                        ip_match = re.search(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}', line)
                        if ip_match:
                            ip = ip_match.group()
                            self.input_ip.setText(ip)
                            self.label_status.setText(f"✅ 抓到 IP: {ip}")
                            self.append_log(f"✅ 自動偵測到 IP: {ip}")
                    
                    # 舊版相容（Camera Ready）
                    elif "Camera Ready! Use 'http://" in line:
                        try:
                            ip = line.split("http://")[1].split(":")[0]
                            self.input_ip.setText(ip)
                            self.label_status.setText(f"✅ 抓到 IP: {ip}")
                        except:
                            pass
        
        except serial.SerialException as e:
            # Port 被佔用或斷線，暫時停用
            if "ClearCommError" in str(e) or "PermissionError" in str(e):
                self.serial_enabled = False
                self.close_serial_safe()
                self.append_log("⚠️  Serial 暫時無法存取（可能正在上傳），稍後自動重試...")
                # 5 秒後重試
                QTimer.singleShot(5000, self.retry_serial)
        
        except Exception as e:
            # 其他未預期錯誤
            pass

    def retry_serial(self):
        """重試開啟 Serial"""
        if not self.serial_enabled:
            self.serial_enabled = True
            self.reopen_serial()

    def closeEvent(self, event):
        """視窗關閉時清理資源"""
        if self.video_thread: 
            self.video_thread.stop()
        self.close_serial_safe()
        event.accept()

if __name__ == "__main__":
    app = QApplication.instance() or QApplication(sys.argv)
    w = Esp32CamWindow()
    w.show()
    sys.exit(app.exec())

SystemExit: 0

C:\Users\ken\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
import sys
import os
import shutil
import time
import subprocess
import re
import cv2
import numpy as np

import serial
from serial.tools import list_ports
from PySide6.QtGui import QTextCursor, QImage, QPixmap
from PySide6.QtCore import QTimer, Qt, QThread, Signal
from PySide6.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QHBoxLayout,
    QLabel, QPushButton, QPlainTextEdit, QMessageBox, 
    QComboBox, QGroupBox, QLineEdit, QSizePolicy
)

os.environ["PYTHONIOENCODING"] = "utf-8"

# ========= 參數設定 =========
SKETCH_DIR = "esp32s3_cam"          
SKETCH_NAME = "esp32s3_cam.ino"
SOURCE_INO = "code.ino"             
FQBN = (
    "esp32:esp32:esp32s3:"
    "FlashSize=16M,"          
    "PSRAM=opi,"              
    "PartitionScheme=fatflash,"  
    "UploadSpeed=921600"      
)
BAUD_RATE = 115200                  
ESP32_URL = "https://espressif.github.io/arduino-esp32/package_esp32_index.json"

# === [FIXED] 影像接收執行緒 ===
class VideoThread(QThread):
    change_pixmap_signal = Signal(QImage)
    status_signal = Signal(str) # 新增狀態回報信號
    
    def __init__(self, url):
        super().__init__()
        self.url = url
        self.is_running = True

    def run(self):
        print(f"嘗試連接串流: {self.url}")
        cap = cv2.VideoCapture(self.url)
        
        if not cap.isOpened():
            self.status_signal.emit("❌ OpenCV 無法開啟串流，請檢查 IP 或防火牆")
            self.is_running = False
            return

        self.status_signal.emit("✅ 串流已連接，開始接收...")

        # 設定 Buffer 為 1 降低延遲
        try:
            cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
        except:
            pass 
        
        fail_count = 0

        while self.is_running:
            ret, cv_img = cap.read()
            if ret:
                fail_count = 0
                # 轉色 BGR -> RGB
                rgb_image = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
                h, w, ch = rgb_image.shape
                bytes_per_line = ch * w
                
                # ★★★ 關鍵修復 ★★★ 
                # 必須加上 .copy()，否則 rgb_image 在下一圈迴圈被釋放時，
                # QImage 指向的記憶體會失效，導致 UI 全黑或崩潰。
                qt_image = QImage(rgb_image.data, w, h, bytes_per_line, QImage.Format_RGB888).copy()
                
                self.change_pixmap_signal.emit(qt_image)
            else:
                fail_count += 1
                if fail_count > 100:
                    self.status_signal.emit("⚠️ 訊號丟失，重連中...")
                    fail_count = 0
                self.msleep(10) # 讀取失敗時稍微休息
        
        cap.release()
        print("串流執行緒結束")

    def stop(self):
        self.is_running = False
        self.wait()

# === 燒錄相關函式 (保持不變) ===
def prepare_sketch():
    if not os.path.exists(SOURCE_INO):
        print(f"❌ 找不到 {SOURCE_INO}，請把 code.ino 放在同一個資料夾。")
        return False
    if not os.path.exists(SKETCH_DIR):
        os.makedirs(SKETCH_DIR, exist_ok=True)
    dest_path = os.path.join(SKETCH_DIR, SKETCH_NAME)
    shutil.copy(SOURCE_INO, dest_path)
    return True

def check_and_install_core(log_func=print):
    log_func("🔍 檢查 ESP32 核心安裝狀態...")
    check = subprocess.run(["arduino-cli", "core", "list"], capture_output=True, text=True, encoding="utf-8", errors="ignore")
    if "esp32:esp32" in check.stdout:
        log_func("✅ ESP32 核心已安裝。")
        return True
    log_func("⚠️ 未偵測到 ESP32 核心，開始自動安裝...")
    subprocess.run(["arduino-cli", "core", "update-index", "--additional-urls", ESP32_URL])
    install = subprocess.run(["arduino-cli", "core", "install", "esp32:esp32", "--additional-urls", ESP32_URL])
    return install.returncode == 0

def compile_and_upload(port, log_func=print):
    if not check_and_install_core(log_func): return False
    log_func("=== Compile (編譯中)... ===")
    comp = subprocess.run(["arduino-cli", "compile", "--fqbn", FQBN, SKETCH_DIR, "--additional-urls", ESP32_URL], capture_output=True, text=True, encoding="utf-8", errors="ignore")
    if comp.returncode != 0:
        log_func("❌ compile 失敗:\n" + comp.stderr)
        return False
    log_func(f"=== Upload (上傳至 {port})... ===")
    upl = subprocess.run(
        ["arduino-cli", "upload", "-p", port, "--fqbn", FQBN, 
         "--upload-field", "upload.speed=115200", "--upload-field", "upload.flash_mode=dio", SKETCH_DIR],
        capture_output=True, text=True, encoding="utf-8", errors="ignore"
    )
    log_func("upload stdout:\n" + upl.stdout)
    if upl.returncode != 0:
        log_func("❌ upload 失敗:\n" + upl.stderr)
        return False
    log_func("✅ Upload complete.")
    return True

# === 主視窗 ===
class Esp32CamWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.ser = None
        self.current_port = None
        self.video_thread = None
        self.serial_enabled = True 
        
        self.init_ui()
        self.refresh_ports()
        self.init_timer()

    def init_ui(self):
        self.setWindowTitle("ESP32-S3-CAM N16R8 控制中心 v7.0")
        layout = QVBoxLayout()

        # 1. Port 選擇
        port_group = QGroupBox("1. 硬體連線")
        port_layout = QHBoxLayout()
        self.combo_ports = QComboBox()
        self.combo_ports.currentIndexChanged.connect(self.on_port_changed)
        self.btn_refresh = QPushButton("🔄 重新整理")
        self.btn_refresh.clicked.connect(self.refresh_ports)
        port_layout.addWidget(self.combo_ports, 1)
        port_layout.addWidget(self.btn_refresh)
        port_group.setLayout(port_layout)

        # 2. 燒錄控制
        upload_group = QGroupBox("2. 韌體燒錄")
        upload_layout = QHBoxLayout()
        self.btn_reset = QPushButton("⚡ 強制 Boot")
        self.btn_reset.clicked.connect(self.force_bootloader)
        self.btn_upload = QPushButton("🔥 上傳韌體")
        self.btn_upload.setStyleSheet("background-color: #d9534f; color: white; font-weight: bold;")
        self.btn_upload.clicked.connect(self.on_upload_clicked)
        upload_layout.addWidget(self.btn_reset)
        upload_layout.addWidget(self.btn_upload)
        upload_group.setLayout(upload_layout)

        # 3. 影像顯示區
        video_group = QGroupBox("3. 即時影像 (GPU 縮放)")
        video_layout = QVBoxLayout()
        
        ip_layout = QHBoxLayout()
        self.input_ip = QLineEdit()
        self.input_ip.setPlaceholderText("等待 Serial 自動抓取 IP 或手動輸入...")
        self.btn_connect = QPushButton("📺 連線影像")
        self.btn_connect.setStyleSheet("background-color: #5bc0de; color: white; font-weight: bold;")
        self.btn_connect.clicked.connect(self.start_video)
        ip_layout.addWidget(QLabel("ESP32 IP:"))
        ip_layout.addWidget(self.input_ip)
        ip_layout.addWidget(self.btn_connect)

        self.video_label = QLabel("等待影像連線...")
        self.video_label.setAlignment(Qt.AlignCenter)
        self.video_label.setStyleSheet("background-color: #222; color: #aaa; border: 2px solid #555;")
        
        # 影像縮放設定
        self.video_label.setScaledContents(True)
        self.video_label.setMinimumSize(320, 240)
        self.video_label.setSizePolicy(QSizePolicy.Ignored, QSizePolicy.Ignored)

        video_layout.addLayout(ip_layout)
        video_layout.addWidget(self.video_label, 1)
        video_group.setLayout(video_layout)

        # 4. 日誌
        self.label_status = QLabel("狀態：就緒")
        self.text_log = QPlainTextEdit()
        self.text_log.setReadOnly(True)
        self.text_log.setMaximumHeight(150)

        layout.addWidget(port_group)
        layout.addWidget(upload_group)
        layout.addWidget(video_group, 1)
        layout.addWidget(self.label_status)
        layout.addWidget(self.text_log)
        self.setLayout(layout)
        self.resize(800, 750)

    def append_log(self, msg: str):
        self.text_log.appendPlainText(msg.rstrip("\n"))
        self.text_log.moveCursor(QTextCursor.End)

    def refresh_ports(self):
        self.close_serial_safe()
        self.combo_ports.blockSignals(True)
        self.combo_ports.clear()
        ports = list_ports.comports()
        best_index = -1
        for i, p in enumerate(ports):
            self.combo_ports.addItem(f"{p.device} - {p.description}")
            if "COM7" in p.device: best_index = i
            elif best_index == -1 and ("ESP32" in p.description.upper() or "CP210" in p.description): best_index = i
        self.combo_ports.blockSignals(False)
        if ports:
            self.combo_ports.setCurrentIndex(best_index if best_index != -1 else 0)
            self.on_port_changed()
        else:
            self.label_status.setText("狀態：未偵測到 Port")

    def on_port_changed(self):
        text = self.combo_ports.currentText()
        if text:
            self.current_port = text.split(" - ")[0]
            self.label_status.setText(f"已選擇 {self.current_port}")
            self.reopen_serial()

    def force_bootloader(self):
        if not self.current_port: return
        self.close_serial_safe()
        try:
            s = serial.Serial(self.current_port, BAUD_RATE)
            s.dtr = False; s.rts = False
            s.dtr = True; s.rts = False; time.sleep(0.1)
            s.rts = True; time.sleep(0.1)
            s.rts = False; time.sleep(0.2)
            s.dtr = False; s.close()
            self.append_log("✅ 強制 Boot 訊號已發送")
        except Exception as e:
            self.append_log(f"⚠️ Boot 訊號發送失敗: {e}")
        time.sleep(0.5)
        self.reopen_serial()

    def on_upload_clicked(self):
        if not self.current_port or not prepare_sketch(): return
        self.serial_enabled = False
        self.close_serial_safe()
        self.append_log("🔒 已關閉 Serial 監視器，準備上傳...")
        QApplication.processEvents()
        time.sleep(1.0)
        ok = compile_and_upload(self.current_port, log_func=self.append_log)
        time.sleep(3.0)
        self.serial_enabled = True
        self.reopen_serial()
        if ok: 
            self.label_status.setText("✅ 上傳成功！正在重啟...")
            self.append_log("\n✅ 韌體已更新，請觀察下方 Log...")
            QMessageBox.information(self, "成功", "✅ 上傳成功！\nESP32 正在重啟中...")
        else:
            self.label_status.setText("❌ 上傳失敗")

    def start_video(self):
        ip = self.input_ip.text().strip()
        if not ip: 
            QMessageBox.warning(self, "提示", "請輸入 IP 或等待自動偵測")
            return
        
        # 確保 URL 正確
        if not ip.startswith("http"):
            url = f"http://{ip}:81/stream"
        else:
            url = ip

        self.append_log(f"📺 正在連接: {url}")
        
        if self.video_thread: 
            self.video_thread.stop()
        
        self.video_thread = VideoThread(url)
        self.video_thread.change_pixmap_signal.connect(self.update_image)
        self.video_thread.status_signal.connect(self.update_video_status)
        self.video_thread.start()

    def update_image(self, img):
        self.video_label.setPixmap(QPixmap.fromImage(img))

    def update_video_status(self, msg):
        self.append_log(msg)
        if "❌" in msg:
            self.video_label.setText(msg)

    def close_serial_safe(self):
        try: 
            if self.ser and self.ser.is_open: self.ser.close()
        except: pass
        finally: self.ser = None

    def reopen_serial(self):
        if not self.current_port or not self.serial_enabled: return
        try:
            if self.ser and self.ser.is_open: self.ser.close()
            self.ser = serial.Serial(self.current_port, BAUD_RATE, timeout=0.1)
            self.append_log(f"🔗 Serial 已連線: {self.current_port}")
        except: self.ser = None

    def init_timer(self):
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.read_serial)
        self.timer.start(100)

    def read_serial(self):
        if not self.serial_enabled or not self.ser: return
        try:
            if self.ser.is_open and self.ser.in_waiting:
                line = self.ser.readline().decode(errors="ignore").strip()
                if line: 
                    self.append_log(f"[ESP]: {line}")
                    # IP 偵測
                    if "IP" in line and ("192." in line or "10." in line or "172." in line):
                        ip_match = re.search(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}', line)
                        if ip_match:
                            ip = ip_match.group()
                            self.input_ip.setText(ip)
                            self.append_log(f"✅ 自動偵測到 IP: {ip}")
        except: pass

    def closeEvent(self, event):
        if self.video_thread: self.video_thread.stop()
        self.close_serial_safe()
        event.accept()

if __name__ == "__main__":
    app = QApplication.instance() or QApplication(sys.argv)
    w = Esp32CamWindow()
    w.show()
    sys.exit(app.exec())

SystemExit: 0

C:\Users\ken\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
